# Experiment on Newsroom metrics

In [1]:
import gc
import os
import time

import polars as pl


def collect(*, verbose=True):
    if verbose:
        print('garbage collector collected %d objects' % gc.collect())
    else:
        gc.collect()

collect()

garbage collector collected 0 objects


In [2]:
news = pl.read_parquet('private-data/filtered_pairs_2022.parquet')

print(len(news), 'rows')

news.head(3)

5946 rows


pair_id,summary_id,article_id,summary,article,simhash_distance,summary_title,article_title
u32,i32,i32,str,str,i64,str,str
1093692,43423102,43422899,"""Tại phiên họp thứ 14 ngày 11/8…","""100% Ủy viên Ủy ban Thường vụ …",16,"""Quốc hội thông qua việc thành …","""Thành lập thị xã Chơn Thành th…"
1718596,43370015,43360018,"""Tập đoàn Điện lực Việt Nam (EV…","""Theo thông tin từ Tập đoàn Điệ…",17,"""Xuất hiện trang web giả mạo th…","""Xuất hiện trang web giả mạo th…"
1258448,44012345,44017658,"""Mưa lớn vừa qua khiến hàng tấn…","""Trận mưa lớn, liên tục trong c…",14,"""Cảnh tan hoang ở nghĩa trang l…","""Xót xa cảnh nghìn ngôi mộ tại …"


In [3]:
import torch


device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device: "%s"' % device)
if torch.cuda.is_available():
    print('Device name: "%s"' % torch.cuda.get_device_name(device))

Device: "cuda"
Device name: "NVIDIA GeForce RTX 3050 Laptop GPU"


## Using Markdown programmatically

In [4]:
from IPython.display import Markdown, display


display(Markdown('*hello*, **world**!'))

*hello*, **world**!

## Newsroom Fragments

In [5]:
from newsroom.analyze import VietnameseFragmentsBatch


sample = news.sample(5, seed=2025)
pair_ids = sample.select('pair_id').to_series().to_list()
summaries = sample.select('summary').to_series().to_list()
articles = sample.select('article').to_series().to_list()

now = time.time()
fragments_batch = VietnameseFragmentsBatch(summaries, articles, device=device,
                                           case=False, batch_size=128)
print('delta (seconds):', round(time.time() - now, 4))

print('pair_id', pair_ids)
s = f'''s
| Summary | Article | Density |
| ------- | ------- | ------- |'''
for item_i in range(len(fragments_batch.fragments)):
    s += '\n'
    s += ' | ' + ' '.join(fragments_batch.fragments[item_i]._norm_summary)
    s += ' | ' + ' '.join(fragments_batch.fragments[item_i]._norm_text)
    # s += f' | {fragments_batch.fragments[item_i].coverage():.4f}'
    s += f' | {fragments_batch.fragments[item_i].density():.4f}'
    # s += f' | {fragments_batch.fragments[item_i].compression():.4f}'
    s += ' |'

display(Markdown(s))

delta (seconds): 3.8307
pair_id [729857, 61706, 1210183, 847534, 32098]


s
| Summary | Article | Density |
| ------- | ------- | ------- |
 | bộ y_tế đề_nghị các cán_bộ nhân_viên có liên_quan đến công_tác cấp hộ_chiếu vắc - xin , không được cản_trở , gây khó_khăn , trục_lợi , phải đảm_bảo quyền_lợi người_dân . | các tổ_chức , cá_nhân không được gây khó_khăn cho người_dân trong cấp chứng_nhận tiêm_chủng và hộ_chiếu vắc_xin ; chịu hoàn_toàn trách_nhiệm trước pháp_luật nếu để xảy ra tiêu_cực . hôm_nay 1 . 5 , bộ y_tế đã có công_văn 2228 gửi các sở y_tế , tập_đoàn công_nghiệp , viễn_thông quân_đội về cấp hộ_chiếu vắc_xin covid - 19 công_văn 2228 nêu rõ , ngày 19 . 4 . 2022 , bộ y_tế đã ban_hành công_văn số 1975 / byt - cntt về việc “ làm sạch ” dữ_liệu tiêm_chủng covid - 19 và triển_khai ký xác_nhận hộ_chiếu vắc_xin . quy_trình cấp hộ_chiếu vắc_xin covid - 19 byt để đảm_bảo quyền của người_dân được cấp giấy chứng_nhận đã tiêm vắc_xin và hộ_chiếu vắc_xin khi tham_gia tiêm_chủng covid - 19 , bộ y_tế đề_nghị các đơn_vị thực_hiện nghiêm việc quán_triệt nhân_viên y_tế và các cán_bộ thuộc quyền quản_lý của đơn_vị có liên_quan đến công_tác ký điện_tử chứng_nhận tiêm_chủng covid - 19 và hộ_chiếu vắc_xin không được cản_trở , gây khó_khăn và có các hành_vi trục_lợi trong việc hỗ_trợ , đảm_bảo quyền_lợi người_dân được cấp chứng_nhận tiêm_chủng covid - 19 và xác_nhận hộ_chiếu vắc_xin . lãnh_đạo bộ y_tế giao vụ truyền_thông và thi_đua khen_thưởng chủ_động thực_hiện truyền_thông tới toàn dân được biết về quyền được cấp giấy chứng_nhận tiêm_chủng và hộ_chiếu vắc_xin khi đã tiêm vắc_xin phòng covid - 19 . bộ y_tế quán_triệt tới tất_cả các tổ_chức / cá_nhân có liên_quan không được gây khó_khăn cho người_dân trong việc cấp chứng_nhận tiêm_chủng và xác_nhận hộ_chiếu vắc_xin , nếu để xảy ra các biểu_hiện tiêu_cực thì phải chịu hoàn_toàn trách_nhiệm trước pháp_luật . quy_trình cấp hộ_chiếu vắc_xin cho người_dân gồm 3 bước bước 1 : các cơ_sở tiêm_chủng rà_soát , xác_minh , xác_thực thông_tin người_dân tiêm_chủng vắc_xin covid - 19 . bước 2 : các cơ_sở tiêm_chủng thực_hiện ký số dữ_liệu tiêm_chủng vắc_xin covid - 19 trên nền_tảng quản_lý tiêm_chủng covid - 19 . nền_tảng quản_lý tiêm_chủng covid - 19 kết_nối , chia_sẻ dữ_liệu tiêm_chủng với hệ_thống quản_lý cấp chứng_nhận tiêm_chủng vắc_xin covid - 19 đáp_ứng theo các quy_định về kết_nối dữ_liệu y_tế do bộ y | 3.0000 |
 | sáng 21 - 4 , ubnd tỉnh thanh_hóa trọng_thể tổ_chức lễ kỷ_niệm 700 năm ngày mất nhà_sử_học lê_văn_hưu ( 1322 - 2022 ) và khánh_thành đền thờ lê_văn_hưu tại xã thiệu_trung , huyện thiệu_hóa . | khẳng_định , tôn_vinh vai_trò và công_lao của nhà_sử_học lê_văn_hưu đối_với nền sử_học và sự phát_triển của dân_tộc việt_nam , sáng nay 21 - 4 , tỉnh thanh_hóa đã long_trọng tổ_chức lễ kỷ_niệm 700 năm ngày mất và khánh_thành đền thờ ông . sáng ngày 21 - 4 , ubnd tỉnh thanh_hóa đã tổ_chức trọng_thể lễ kỷ_niệm 700 năm ( 1322 - 2022 ) ngày mất nhà_sử_học lê_văn_hưuvà khánh_thành đền thờ ông tại xã thiệu_trung , huyện thiệu_hóa , tỉnh thanh_hóa . ông đầu_thanh_tùng , phó chủ_tịch ubnd tỉnh thanh_hóa , đọc diễn_văn tại lễ kỷ_niệm theo tài_liệu lịch_sử , lê_văn_hưu sinh năm canh_dần ( 1230 ) trong một dòng_họ nổi_tiếng ở làng phủ_lý , nay là xã thiệu_trung , huyện thiệu_hóa ( thanh_hóa ) . 17 tuổi lê_văn_hưu thi đậu bảng_nhãn trong khoa_thi đầu_tiên của nhà trần . lê_văn_hưu từng được giao nhiều chức_vụ quan_trọng của triều_đình nhà trần , như : kiểm pháp_quan , hàn_lâm_viện_học_sĩ kiêm quốc_sử viện tu giám , làm phó quan cho thượng_tướng_quân trần_quang_khải . các đại_biểu dự lễ kỷ_niệm lúc sinh_thời , nhà_sử_học lê_văn_hưu là người có lòng yêu_thương dân_chúng và cống_hiến , phụng_sự triều_đình , với mong_muốn góp_phần xây_dựng một xã_hội hài_hòa , đất_nước thái_bình , thịnh_vượng . ông mất năm nhâm_tuất ( 1322 ) , hưởng_thọ 93 tuổi . hiện_nay , trên đất thiệu_trung vẫn còn phần_mộ , bia ký khắc ghi tiểu_sử , ca_tụng tài_đức , sự_nghiệp của ông . đóng_góp lớn nhất , và đã đưa tên_tuổi của lê_văn_hưu vào lịch_sử dân_tộc không dừng lại ở việc đỗ bảng_nhãn và làm quan mà là khi ông được vua trần_thái_tông điều_chuyển sang làm hàn_lâm_viện_học_sĩ kiêm quốc_sử viện giám_tu . thực_hiện lệnh của nhà_vua , lê_văn_hưu đã thu_thập tất_cả các sách sử ghi_chép ít_ỏi và sơ_sài của thời lý và cùng thời để biên_soạn lại và viết thêm rất nhiều để thành bộ quốc_sử có tên " đại_việt_sử_ký " , gồm 30 quyển . hàng ngàn người_dân địa_phương đã tới dự lễ kỷ_niệm 700 năm ngày mất nhà_sử_học lê_văn_hưu " đại_việt_sử_ký " của lê_văn_hưu như dấu mốc lớn đánh_dấu thành_tựu khoa_học , đặt cơ_sở đầu_tiên cho nền sử_học nước_nhà phát_triển . lê_văn | 5.2286 |
 | gs . tskh nguyễn_đình_đức , trưởng ban đào_tạo , đh quốc_gia hà_nội , nhận_định điểm chuẩn năm nay ở các ngành của trường sẽ tăng 0 , 5 - 1 điểm ; những ngành hot tăng từ 1 đến 2 điểm . | trong buổi tư_vấn tuyển_sinh trực_tuyến vừa_qua , gs . tskh nguyễn_đình_đức , trưởng ban đào_tạo , đại_học quốc_gia hà_nội dự_đoán điểm trúng_tuyển ở các ngành đào_tạo theo phương_thức xét điểm thi tốt_nghiệp thpt năm 2022 của đại_học này sẽ tăng . tại buổi tư_vấn trực_tuyến , dựa trên phổ điểm các môn thi tốt_nghiệp thpt do bộ gd & đt công_bố , gs . tskh nguyễn_đình_đức nhận_định tỷ_lệ thí_sinh đạt điểm 8 trở_lên ở môn tiếng anh đã giảm gần một_nửa so với năm trước đó . cụ_thể , năm nay cả nước có 11 , 3 % thí_sinh đạt điểm 8 trở_lên ở môn thi này . trong khi đó , năm_ngoái số_lượng thí_sinh đạt điểm 8 trở lên là gần 20 % . theo ông đức , những tổ_hợp có môn tiếng anh sẽ hạ điểm chuẩn . ngược_lại , ở các tổ_hợp có môn thi lịch_sử , gs đức nhận_định điểm chuẩn sẽ tăng nhẹ . nguyên_nhân là môn thi này có 18 % thí_sinh đạt điểm 8 trở_lên , cao hơn trước rất nhiều . đồng_thời cho biết thêm năm nay chỉ_tiêu ưu_tiên xét tuyển thẳng và tuyển thẳng của đh quốc_gia hà_nội vẫn cao . các trường trực_thuộc đh cũng tăng chỉ_tiêu xét tuyển bằng điểm thi đánh_giá năng_lực nên chỉ_tiêu còn lại cho phương_thức xét điểm thi tốt_nghiệp thpt năm 2022 thấp hơn . vì_vậy , điểm chuẩn của các ngành ở phương_thức này sẽ tăng thêm từ 0 , 5 đến 1 điểm . đối_với những ngành hot , gs nguyễn_đình_đức dự_đoán điểm chuẩn cao hơn từ 1 đến 2 điểm so với năm 2021 . trong đó , các lĩnh_vực khoa_học_xã_hội của đh quốc_gia hà_nội năm nay sẽ có điểm chuẩn cao . khối khoa_học_tự_nhiên giữ ổn_định điểm chuẩn như năm_ngoái . qua các năm tuyển_sinh , gs nguyễn_đình_đức nhận_định những ngành hot trong xu_hướng nghề_nghiệp là kỹ_thuật công_nghệ ( lĩnh_vực thời_thượng của thời_đại , đáp_ứng nhu_cầu nhân_lực của cuộc cách_mạng công_nghiệp 4_. 0 ) ; y , dược , luật , kinh_tế ( những ngành truyền_thống vẫn giữ được độ hot từ trước đến nay ) ; và các ngành liên_ngành như khoa_học_dữ_liệu , điện_tử tin_học , khoa_học_máy_tính , hàn_quốc_học … | 4.0238 |
 | theo công_an tp hcm , bà nguyễn_phương_hằng đã tự_nguyện cho ông võ_hoàng_yên số tiền 183 tỷ đồng để ông yên trả nợ , xây_dựng , sửa_chữa chùa , làm từ_thiện . | cơ_quan công_an kết_luận bà phương_hằng tố ông võ_hoàng_yên không có dấu_hiệu tội_phạm . dư_luận đặt câu hỏi : liệu ' gậy ông có đập lưng bà ' ? . mới_đây , cơ_quan csđt công_an tp hcm đã ra quyết_định không khởi_tố vụ án hình_sự đối_với đơn tố_giác về tội_phạm của bà nguyễn_phương_hằng đối_với ông võ_hoàng_yên . viện ksnd tp hcm cũng có kết_luận quyết_định của cơ_quan điều_tra là có căn_cứ , đúng pháp_luật . lý_do không khởi_tố vụ án do cơ_quan điều_tra xác_định việc bà hằng tố_cáo võ_hoàng_yên không có dấu_hiệu tội_phạm . quá_trình điều_tra không có căn_cứ chứng_minh việc ông yên đưa ra thông_tin gian_dối để bà hằng tin_tưởng mà giao tiền , không có căn_cứ chứng_minh việc ông yên sử_dụng số tiền vào mục_đích bất_hợp_pháp , gian_dối nhằm chiếm_đoạt . số tiền 183 tỷ đồng , trong đó có 60 tỷ tiền_mặt , bà hằng tự_nguyện đưa cho ông yên để trả nợ và xây_dựng , sửa_chữa chùa , làm từ_thiện . ngoài tố_cáo ông võ_hoàng_yên về hành_vi lừa_đảo chiếm_đoạt tài_sản , không minh_bạch các hoạt_động từ_thiện , xây chùa , ủng_hộ bão_lụt … thì bà hằng còn điểm tên nhiều nghệ_sĩ . bà nguyễn_phương_hằng nhiều lần livestream nói về việc các cá_nhân , ca_sĩ , nghệ_sĩ kêu_gọi và ăn_chặn tiền từ_thiện như : hoài_linh , ca_sĩ đàm_vĩnh_hưng , vợ_chồng ca_sĩ thủy_tiên - công_vinh , mc trấn_thành . . . trong livestream , bà phương_hằng đề_cập việc nghệ_sĩ hoài_linh chưa thực_hiện cứu_trợ miền trung dù đã nhận hơn 13 tỷ quyên_góp . một_số cá_nhân sau đó tố_cáo nghệ_sĩ hoài_linh lạm_dụng tín_nhiệm chiếm_đoạt tài_sản . tuy_nhiên , kết_quả điều_tra xác_minh nội_dung tố_cáo , cơ_quan csđt công_an tphcm xác_định không có sự_việc phạm_tội , nên không khởi_tố vụ án hình_sự . cục cảnh_sát hình_sự ( c02 ) bộ công_an xác_định các cá_nhân là các nghệ_sĩ có kêu_gọi từ_thiện ủng_hộ bão_lụt , có công_khai tài_khoản để tiền các cá_nhân tổ_chức gửi vào tài_khoản đó . sau một thời_gian nhất_định , họ đã đóng tài_khoản và dừng kêu_gọi . khi có lượng tiền vào tài_khoản , họ trực_tiếp hoặc thông_qua đại_diện đi địa_phương làm từ_thiện . c02 có căn_cứ xác_định , lượng tiền vào tài | 3.8710 |
 | sáng 11 / 1 , chịu ảnh_hưởng của không_khí lạnh , khu_vực hà_nội nhiệt_độ giảm sâu , ngoài_trời chỉ khoảng hơn 10 độ c , những người đi làm sớm trang_bị nhiều lớp quần_áo dày kín_mít để chống rét . | trong ngày 11 / 1 , do ảnh_hưởng của gió_mùa đông_bắc nên các tỉnh bắc_bộ có mưa và rét trên diện rộng . nhiệt_độ thấp nhất có nơi dưỡi 11 độ c . ảnh minh_họa . tin gió_mùa đông_bắc trung_tâm dự_báo khí_tượng_thủy_văn quốc_gia cho biết , đêm hôm_qua ( 10 / 1 ) , không_khí lạnh đã ảnh_hưởng đến một_số nơi ở khu_vực đông_bắc bắc_bộ . ở vịnh bắc_bộ đã có gió đông_bắc mạnh cấp 6 , giật cấp 8 , biển động . dự_báo , trong ngày hôm_nay ( 11 / 1 ) , không_khí lạnh tiếp_tục ảnh_hưởng đến các nơi khác ở bắc_bộ và bắc trung_bộ , sau đó ảnh_hưởng đến một_số nơi thuộc trung_trung_bộ . do ảnh_hưởng của không_khí lạnh nên sáng nay ( 11 / 1 ) , ở bắc_bộ có mưa và mưa nhỏ rải_rác ; bắc_trung_bộ và trung trung_bộ có mưa , mưa_rào và có nơi có dông , cục_bộ có mưa vừa , mưa to . trong mưa_dông có khả_năng xảy ra lốc , sét , mưa_đá và gió_giật mạnh . ở bắc_bộ và bắc trung_bộ trời rét , vùng núi có nơi rét đậm . nhiệt_độ thấp nhất ở bắc_bộ phổ_biến 12 - 15 độ , vùng_núi có nơi dưới 11 độ ; bắc trung_bộ phổ_biến 14 - 17 độ . trong đất_liền gió đông_bắc mạnh cấp 3 , vùng_ven biển cấp 4 ; ở vịnh bắc_bộ gió đông_bắc mạnh cấp 6 , giật cấp 7 - 8 , biển động , sóng biển cao từ 2 , 0 - 3 , 0m . khu_vực bắc biển_đông ( bao_gồm cả vùng_biển quần_đảo hoàng_sa ) , vùng_biển từ bình_định đến cà_mau , khu_vực giữa biển_đông và vùng_biển phía tây khu_vực nam_biển_đông ( bao_gồm cả vùng_biển phía tây quần_đảo trường_sa ) gió đông_bắc mạnh cấp 6 , giật cấp 7 - 8 , biển động , sóng biển cao từ 2 , 0 - 4 , 0m . dự_báo thời_tiết các vùng trên cả nước hà_nội nhiệt_độ thấp nhất : 12 - 15 độ . nhiệt_độ cao nhất : 15 - 18 độ . nhiều_mây , sáng có mưa nhỏ rải_rác , sau có mưa nhỏ vài nơi . gió đông_bắc cấp 3 . trời rét . phía tây bắc_bộ nhiệt_độ thấp nhất : 12 - 15 độ , có nơi dưới 11 độ . nhiệt_độ cao nhất : 16 - 19 độ , riêng khu_vực tây_bắc có nơi trên 20 độ . nhiều_mây , sáng có | 1.3000 |